In [3]:
! pip install pytorch-lightning

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [4]:
! pip install tensorboard

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import lightning as L
import torchmetrics
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,random_split
from torchvision.transforms import ToTensor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
logger = TensorBoardLogger("tb_logs", name="basic_cnn_model")

## Create Basic CNN model

In [7]:
class CNNModel(L.LightningModule):
    def __init__(self, input_shape,hidden_unit,output_shape=10):
        super(CNNModel,self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels = input_shape,
                      out_channels = hidden_unit,
                      kernel_size=3,
                      stride =1,
                      padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )

        self.conv_block2 = nn.Sequential(    
            nn.Conv2d(in_channels = hidden_unit,
                      out_channels = hidden_unit,
                      kernel_size=3,
                      stride =1,
                      padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )

        self.conv_block3 = nn.Sequential( 
             nn.Conv2d(in_channels = hidden_unit,
                      out_channels = hidden_unit,
                      kernel_size=3,
                      stride =1,
                      padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
            
        self.estimator = nn.Sequential(
            nn.Linear(hidden_unit*20 * 20, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, output_shape)
        )

        self.accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=output_shape)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = x.view(x.shape[0], -1)

        return self.estimator(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc =self.accuracy(y_hat, y)
        self.log("val_accuracy", self.accuracy, prog_bar=True, logger=True)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        acc=self.accuracy(y_hat, y)

        self.log("val_accuracy", self.accuracy, prog_bar=True, logger=True)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return  loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)

        acc = self.accuracy(y_hat, y)

        self.log("test_accuracy", self.accuracy, prog_bar=True, logger=True)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return  loss,acc

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


## Load data and Preprocessing Data

load data and augmentation it by rotation,translation and Flip

In [8]:
from pathlib import Path
train_dataset_path= Path("data/imagenette/train")
test_dataset_path= Path("data/imagenette/test")
train_dataset_download=False if train_dataset_path.exists() else True
test_dataset_download=False if test_dataset_path.exists() else True
print(f"test_dataset_download:{train_dataset_download},test_dataset_download:{test_dataset_download}")

train_transforms = transforms.Compose([
    transforms.CenterCrop(320),
    transforms.Resize(32),
    
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(10), 
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), 
    
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    # transforms.Grayscale()
])

test_transforms = transforms.Compose([
    transforms.CenterCrop(320),
    transforms.Resize(32),

    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(10), 
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
    # transforms.Grayscale()
])

train_dataset = datasets.Imagenette(
    root = 'data/imagenette/train/',
    split = 'train',
    size = '320px',
    download = train_dataset_download,
    transform= train_transforms,
    target_transform = None)

test_dataset = datasets.Imagenette(
    root = 'data/imagenette/test/',
    split = 'val',
    size = '320px',
    download = test_dataset_download,
    transform= test_transforms,
    target_transform = None)
# Use 10% of the training set for validation
train_set_size = int(len(train_dataset) * 0.9)
val_set_size = len(train_dataset) - train_set_size

seed = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(train_dataset, [train_set_size, val_set_size], generator=seed)
val_dataset.dataset.transform = test_transforms

# Use DataLoader to load the dataset
train_loader = DataLoader(train_dataset, batch_size=128, num_workers=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, num_workers=8, shuffle=False)

train_loader,test_dataset,train_loader

test_dataset_download:False,test_dataset_download:False


(<torch.utils.data.dataloader.DataLoader at 0x7f4803910490>,
 Dataset Imagenette
     Number of datapoints: 3925
     Root location: data/imagenette/test/
     StandardTransform
 Transform: Compose(
                CenterCrop(size=(320, 320))
                Resize(size=32, interpolation=bilinear, max_size=None, antialias=True)
                RandomHorizontalFlip(p=0.5)
                RandomRotation(degrees=[-10.0, 10.0], interpolation=nearest, expand=False, fill=0)
                RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
                ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.2, 0.2))
                ToTensor()
                Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.2435, 0.2616))
            ),
 <torch.utils.data.dataloader.DataLoader at 0x7f4803910490>)

In [9]:
train_features_batch,train_labels_batch=next(iter(train_loader))
len(train_dataset),len(test_dataset),len(val_dataset),train_features_batch.shape

(8522, 3925, 947, torch.Size([128, 3, 32, 32]))

In [10]:
cnnm_02 = ALLCNNCModel(input_shape=3,output_shape=10)

# Add EarlyStopping
early_stop_callback = EarlyStopping(monitor="val_loss",
                                    mode="min",
                                    patience=5)

# Configure Checkpoints
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min"
)

In [11]:
# Fit the model
trainer = L.Trainer(callbacks=[early_stop_callback, checkpoint_callback], max_epochs=20)
trainer.fit(model=cnnm_02, train_dataloaders=train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params | Mode 
-----------------------------------------------------------
0 | conv_block1 | Sequential         | 168 K  | train
1 | conv_block2 | Sequential         | 830 K  | train
2 | conv_block3 | Sequential         | 370 K  | train
3 | accuracy    | MulticlassAccuracy | 0      | train
-----------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.479     Total estimated model params size (MB)


Epoch 19: 100%|██████████| 67/67 [00:06<00:00, 10.76it/s, v_num=2, val_accuracy=0.416, train_loss=1.550, val_loss=1.760]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 67/67 [00:06<00:00, 10.66it/s, v_num=2, val_accuracy=0.416, train_loss=1.550, val_loss=1.760]


In [14]:
# Evaluate the model on the test set
test_loader = DataLoader(test_dataset, batch_size=256, num_workers=8, shuffle=False)
trainer.test(model=cnnm_02, dataloaders=test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 16/16 [00:01<00:00, 13.76it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.4091719686985016
        test_loss            1.717761516571045
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 0.4091719686985016, 'test_loss': 1.717761516571045}]

In [13]:
#save model
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,exist_ok = True)
MODEL_NAME = "CNN_regulation_model.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME
torch.save(obj = cnnm_02.state_dict(),
           f=MODEL_SAVE_PATH)